In [100]:
import itertools as it
import functools as ft
import operator
from fractions import Fraction
import math

class CalkinWilf:
    ROOT_LEVEL = 0
    ROOT_POSITION = 0
    ROOT_MOVE_LIST = []
    ROOT_FRACTION_TUPLE = (1, 1)

    def __init__(self, level=None, position=None, move_list=None, fraction_tuple=None):
        if level is None:
            self.level = type(self).ROOT_LEVEL
            self.position = type(self).ROOT_POSITION
            self.bfs_index = 2 ** type(self).ROOT_LEVEL + type(self).ROOT_POSITION
            self.move_list = type(self).ROOT_MOVE_LIST
            self.fraction_tuple = type(self).ROOT_FRACTION_TUPLE
        else:
            self.level = level
            self.position = position
            self.bfs_index = 2 ** self.level + self.position
            self.move_list = move_list
            self.fraction_tuple = fraction_tuple

    @staticmethod
    def cf_to_convergent_list(a: list[int]) -> list[tuple[int, int]]:
        N = len(a)
        if N == 0:
            return []
        p_prev2, p_prev1 = 0, 1
        q_prev2, q_prev1 = 1, 0
        convergent_list = []
        for k in range(N):
            p_k = a[k] * p_prev1 + p_prev2
            q_k = a[k] * q_prev1 + q_prev2
            convergent_list.append((p_k, q_k))
            p_prev2, p_prev1 = p_prev1, p_k
            q_prev2, q_prev1 = q_prev1, q_k        
        return convergent_list

    @staticmethod
    @ft.lru_cache
    def stern_diatomic(n):
        def stern_diatomic_recurse(n):
            if n == 0:
                return 0
            elif n == 1:
                return 1
            else:
                if n % 2 == 0:
                    return stern_diatomic_recurse(n // 2)
                else:
                    m = (n - 1) // 2
                    return stern_diatomic_recurse(m) + stern_diatomic_recurse(m + 1)
        return stern_diatomic_recurse(n)
    
    @staticmethod
    def bfs_index_to_fraction_tuple_recursive(n):
        @ft.lru_cache
        def recurse_q(n):
            if n == 0:
                return Fraction(1, 1)
            else:
                return 1 / (2 * int(recurse_q(n - 1)) - recurse_q(n - 1) + 1)
        q = recurse_q(n - 1)
        return q.as_integer_ratio()
    
    @classmethod
    def move_list_to_fraction_tuple(cls, move_list):
        runs = [(k, len(list(g))) for k, g in it.groupby(move_list)]
        if not runs:
            return (1, 1)
        move_label_tuple, run_length_tuple = zip(*runs)
        move_label_list, run_length_list = list(move_label_tuple), list(run_length_tuple)
        if move_label_list[0] == "L":
            move_label_list.insert(0, "R")
            run_length_list.insert(0, 0)

        convergent_list = cls.cf_to_convergent_list(run_length_list)
        return convergent_list[-1]

    @classmethod
    def bfs_index_to_move_list(cls, n: int) -> list[str]:
        move_decode_dict = {"0": "L", "1": "R"}
        bit_string = f"{n + 1:b}"
        bit_list = list(bit_string)
        move_list = [move_decode_dict[bit] for bit in bit_list]
        return move_list
    
    @classmethod
    def bfs_index_to_fraction_tuple(cls, n: int) -> tuple[int, int]:
        bit_string = f"{n:b}"
        runs = [(bit, len(list(g))) for bit, g in it.groupby(reversed(bit_string))]
        move_label_tuple, run_length_tuple = zip(*runs)
        move_label_list, run_length_list = list(move_label_tuple), list(run_length_tuple)
        if move_label_list[0] == "0":
            run_length_list.insert(0, 0)
            move_label_list.insert(0, "1")
        
        convergent_list = cls.cf_to_convergent_list(run_length_list)
        return convergent_list[-1]


    @classmethod
    def move_list_to_position(cls, move_list: list[str]) -> int:
        if not move_list:
            return cls.ROOT_POSITION
        move_encode_dict = {"L": "0", "R": "1"}
        move_list_encoded = [move_encode_dict[move] for move in move_list]
        move_list_string = "".join(move_list_encoded)
        position = int(move_list_string, base=2)
        return position
    
    @classmethod
    def move_list_to_bfs_index(cls, move_list: list[str]) -> int:
        level = len(move_list)
        position = cls.move_list_to_position(move_list)
        bfs_index = 2 ** level + position
        return bfs_index
    
    @staticmethod
    def fraction_tuple_to_move_list(fraction_tuple):
        def fraction_tuple_to_move_list_recurse(fraction_tuple, move_list):
            if fraction_tuple == (1, 1):
                return []
            else:
                a, b = fraction_tuple
                if a < b:
                    new_move_list = ["L"] + move_list
                    new_fraction_tuple = (a, b - a)
                else:
                    new_move_list = ["R"] + move_list
                    new_fraction_tuple = (a - b, b)
                return fraction_tuple_to_move_list_recurse(new_fraction_tuple, new_move_list)
        return fraction_tuple_to_move_list_recurse(fraction_tuple)

    @classmethod
    def fraction_tuple_to_node(cls, fraction_tuple):
        move_list = cls.fraction_tuple_to_move_list(fraction_tuple, [])
        level = len(move_list)
        position = cls.move_list_to_position(move_list)
        return cls(level, position, move_list, fraction_tuple)
    
    @classmethod
    def depth_to_bfs_node_list(cls, depth):
        cw_tree = []
        N = cls()
        level_node_list = []
        level_node_list.append(N)
        cw_tree.append(level_node_list)
        for n in range(depth):
            previous_level_node_list = level_node_list
            level_node_list = []
            for N in previous_level_node_list:
                level_node_list.append(N.L())
                level_node_list.append(N.R())
            cw_tree.append(level_node_list)
        cw_bfs_list = ft.reduce(operator.iadd, cw_tree, [])
        return cw_bfs_list

    def __repr__(self):
        return f"{type(self).__name__}({self.level}, {self.position}, {self.move_list}, {self.fraction_tuple})"
    
    def __eq__(self, other):
        return self.level == other.level and self.position == other.position
    
    def __lt__(self, other):
        return len(self.move_list) < len(other.move_list) and other.move_list[:len(self.move_list)] == self.move_list
    
    def L(self):
        new_level = self.level + 1
        new_position = 2 * self.position
        new_move_list = self.move_list + ["L"]
        a, b = self.fraction_tuple
        new_fraction_tuple = (a, a + b)
        return type(self)(new_level, new_position, new_move_list, new_fraction_tuple)
    
    def R(self):
        new_level = self.level + 1
        new_position = 2 * self.position + 1
        new_move_list = self.move_list + ["R"]
        a, b = self.fraction_tuple
        new_fraction_tuple = (a + b, b)
        return type(self)(new_level, new_position, new_move_list, new_fraction_tuple)
    
    def P(self):
        if self.level == type(self).ROOT_LEVEL:
            return None
        else:
            new_level = self.level - 1
            new_position = self.position // 2
            move_list = self.move_list
            new_move_list = move_list[0:-1]
            a, b = self.fraction_tuple
            if a < b:
                new_fraction_tuple = (a, b - a)
            else:
                new_fraction_tuple = (a - b, b)
            return type(self)(new_level, new_position, new_move_list, new_fraction_tuple)
    
    def run_tuple_list(self):
        return [(k, len(list(g))) for k, g in it.groupby(self.move_list)]
   



if __name__ == "__main__":
    n = 102

    stern_diatomic_fraction_tuple = (CalkinWilf.stern_diatomic(n + 1), CalkinWilf.stern_diatomic(n + 2))
    bfs_fraction_tuple = CalkinWilf.bfs_index_to_fraction_tuple_recursive(n + 1)
    assert stern_diatomic_fraction_tuple == bfs_fraction_tuple

    bfs_fraction_tuple = CalkinWilf.bfs_index_to_fraction_tuple_recursive(n + 1)
    bfs_index_to_fraction_tuple = CalkinWilf.bfs_index_to_fraction_tuple(n + 1)
    assert bfs_fraction_tuple == bfs_index_to_fraction_tuple
    
    depth = int(math.log2(n))
    cw_bfs_list = CalkinWilf.depth_to_bfs_node_list(depth)
    assert len(cw_bfs_list) > n

    depth = int(math.log2(n))
    cw_bfs_list = CalkinWilf.depth_to_bfs_node_list(depth)
    N = cw_bfs_list[n]
    assert N.bfs_index - 1 == n

    depth = int(math.log2(n))
    cw_bfs_list = CalkinWilf.depth_to_bfs_node_list(depth)
    N1 = cw_bfs_list[n]
    N2 = N.R()
    assert N2 > N1

    depth = int(math.log2(n))
    cw_bfs_list = CalkinWilf.depth_to_bfs_node_list(depth)
    N = cw_bfs_list[n]
    assert CalkinWilf.bfs_index_to_fraction_tuple_recursive(n + 1) == N.fraction_tuple

    depth = int(math.log2(n))
    cw_bfs_list = CalkinWilf.depth_to_bfs_node_list(depth)
    N = cw_bfs_list[n]
    print(CalkinWilf.move_list_to_bfs_index(N.move_list))

103
